In [ ]:
%matplotlib inline
import glob

import numpy as np
import matplotlib.pyplot as plt
import netCDF4
import konrad
import typhon


def scale_getter(filename):
    try:
        return float(konrad.utils.extract_metadata(filename).scale)
    except:
        return ''

    
plt.style.use(typhon.plots.styles('typhon'))
plt.rc('lines', lw=3)

atmosphere = 'rce-rrtmg-nopeak'
experiment = 'scale-co2-nopeak'

filepattern = konrad.utils.get_filepath(
    atmosphere=atmosphere,
    experiment=experiment,
    scale='*',
    extra='',
    create_tree=False,
)
filelist = sorted(glob.glob(filepattern), reverse=True)
filelist = [
    'results/rce-test.nc',
    'results/rce-rrtmg/scale-co2-coupledrh/rce-rrtmg_scale-co2-coupledrh_1.nc',
]

timestep = -1
# Axis settings comparable to Zelinka et al.
# plotfunc = typhon.plots.profile_p
# pmax = 700e2
# pmin = 80e2

plotfunc = typhon.plots.profile_p_log
pmax = 1000e2
pmin = 0.01e2

fig, axes = plt.subplots(nrows=3, ncols=2, sharey=True, figsize=(10, 14))
fig.suptitle(f'Initial: {atmosphere}\n Experiment: {experiment}')
axes = axes.flat
for ncfile in filelist:
    print(ncfile)
    with netCDF4.Dataset(ncfile) as dataset:
        Qr = dataset.variables['net_htngrt'][timestep, :]
        lw_flxd = dataset['lw_flxd'][timestep, :]
        lw_flxu = dataset['lw_flxu'][timestep, :]
        Ts = dataset['temperature'][timestep]
        print(f'Surface temperature {Ts}')

    atm = konrad.atmosphere.Atmosphere.from_netcdf(ncfile, timestep)
    plev = atm['plev'].values
    
    t = atm['T'].values[0, :]
    q = typhon.physics.vmr2specific_humidity(atm['H2O'].values[0, :])
    theta = atm.get_potential_temperature()
    sigma = atm.get_static_stability()
    omega = atm.get_diabatic_subsidence(Qr)
    omega_conv = np.diff(omega) / np.diff(plev[:-1])
    plotlabel = rf'CO$_2$ $\times$ {scale_getter(ncfile)}'

    plotfunc(plev, t, ax=axes[0])
    axes[0].set_xlabel('Temperature [K]', fontsize='small')
    axes[0].set_xlim(170, 310)
   
    plotfunc(plev, q, ax=axes[1])
    axes[1].set_xlabel('Specific humidity [kg/kg]', fontsize='small')
    axes[1].set_xlim(1e-6, 1e-1)
    axes[1].set_xscale('log')
    
    plotfunc(plev[:-1], -Qr[:-1], ax=axes[2])
    axes[2].set_xlabel('Radiative cooling [K/day]', fontsize='small')
    axes[2].set_xlim(-0.5, 3)
    
    plotfunc(plev[:-1], sigma * 100, ax=axes[3])
    axes[3].set_xlim(-0.1, 0.5)
    axes[3].set_xticks(np.arange(-0.1, 0.51, 0.1))
    axes[3].set_xlabel('Static stability [K/hPa]', fontsize='small')

    plotfunc(plev[:-1], omega / 100, ax=axes[4])
    axes[4].set_xlabel('Diabatic $\omega$ [hPa/day]', fontsize='small')
    
    plotfunc(plev[1:-1], omega_conv, ax=axes[5], label=plotlabel)
    axes[5].set_xlabel('Diabatic convergence [1/day]', fontsize='small')
    axes[5].legend()
    
# Remove spines to increase data-ink ratio.
for ax in axes:
    ax.grid(True)
    ax.set_ylim(bottom=pmax, top=pmin)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    if not ax.is_first_col():
        ax.spines['left'].set_visible(False)
        ax.yaxis.set_ticks_position('none')
        
# fig.savefig(f'plots/{atmosphere}_{experiment}_vertical_structure.pdf')

In [ ]:
lim = -0.1

fig, [ax, ax2] = plt.subplots(ncols=2, sharey=True)
ax.axhline(atm.get_convective_top(Qr, lim), color='orangered')
typhon.plots.profile_p(plev, Qr, alpha=0.2, color='k', ax=ax)
typhon.plots.profile_p(plev, Qr, marker='.', color='k', linestyle='none', ax=ax)
ax.grid(True)
ax.set_xlim(left=-2)
ax.set_xlabel('Heatingrate [K/day]')

ax2.axhline(atm.get_convective_top(Qr, lim), color='orangered')
typhon.plots.profile_p(plev[1:-1], omega_conv, alpha=0.2, color='k', ax=ax2)
typhon.plots.profile_p(plev[1:-1], omega_conv, marker='.', linestyle='none', color='k', ax=ax2)
ax2.grid(True)
ax2.set_xlim(left=-9)
ax2.set_xlabel('Diabatic convergence [1/day]')
ax.set_ylim(500e2, 10e2)

for a in [ax, ax2]:
    a.spines['right'].set_visible(False)
    a.spines['top'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.yaxis.set_ticks_position('none')
typhon.plots.label_axes(axes=(ax, ax2), labels=('a)', 'b)'), weight='bold', loc=(0.02, 0.95))

# fig.savefig('plots/convective_top.pdf')